# Exercise 6

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.util import random_noise
from skimage import restoration
from scipy.signal import convolve2d

FS = 15 # Fontsize of caption

## 6.1 Noise mean filtering

In [ ]:
# Read the image test
image = cv2.imread('coins.tif')

# Add gaussian noise to the original image
mean = 0
variance = 0.05
noise = np.random.normal(mean, np.sqrt(variance), image.shape)
noisy_image = cv2.normalize(image + noise * 255, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# Apply an averaging filter with two different kernel sizes
# a. Default kernel size 3x3
kernel_3x3 = np.ones((3, 3), np.float32) / 9
denoised_image_3x3 = cv2.filter2D(noisy_image, -1, kernel_3x3)

# b. Defined kernel size 5x5
kernel_5x5 = np.ones((5, 5), np.float32) / 25
denoised_image_5x5 = cv2.filter2D(noisy_image, -1, kernel_5x5)

# Plot images
plt.figure(figsize=(12, 8))

# Original image
plt.subplot(2, 2, 1)
plt.imshow(image)
plt.title('Original Image', fontsize = FS)
plt.axis('off')

# Noisy image
plt.subplot(2, 2, 2)
plt.imshow(noisy_image)
plt.title('Noisy Image', fontsize = FS)
plt.axis('off')

# Denoised with 3x3 kernel
plt.subplot(2, 2, 3)
plt.imshow(denoised_image_3x3)
plt.title('Denoised with 3x3 Kernel', fontsize = FS)
plt.axis('off')

# Denoised with 5x5 kernel
plt.subplot(2, 2, 4)
plt.imshow(denoised_image_5x5)
plt.title('Denoised with 5x5 Kernel', fontsize = FS)
plt.axis('off')

plt.tight_layout()
plt.show()

## 6.2 Noise salt/pepper filtering

In [ ]:
# Load the original image
image = cv2.imread('coins.tif', cv2.IMREAD_GRAYSCALE)
fs = 15  # Font size for titles

# Add salt and pepper noise to the original image
salt_noise = random_noise(image, mode='salt', amount=0.02)  # Add salt noise
pepper_noise = random_noise(image, mode='pepper', amount=0.02)  # Add pepper noise

# Convert the noisy images to uint8 for processing
salt_noise = (salt_noise * 255).astype(np.uint8)
pepper_noise = (pepper_noise * 255).astype(np.uint8)

# Denoise the images using minimum and maximum filters
# Define the kernel size
kernel_size = (4, 4)

# Remove salt noise using a minimum filter
denoised_salt = cv2.erode(salt_noise, np.ones(kernel_size, np.uint8))

# Remove pepper noise using a maximum filter
denoised_pepper = cv2.dilate(pepper_noise, np.ones(kernel_size, np.uint8))

# Apply max filter to the salt noise image to simulate the "wrong" filter application
wrong_denoised_salt = cv2.dilate(salt_noise, np.ones(kernel_size, np.uint8))

# Plot the results
plt.figure(figsize=(12, 8))

# Original image
plt.subplot(2, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image', fontsize=fs)
plt.axis('off')

# Salt noise image
plt.subplot(2, 3, 2)
plt.imshow(salt_noise, cmap='gray')
plt.title('Salt Noise Image', fontsize=fs)
plt.axis('off')

# Pepper noise image
plt.subplot(2, 3, 3)
plt.imshow(pepper_noise, cmap='gray')
plt.title('Pepper Noise Image', fontsize=fs)
plt.axis('off')

# Salt noise denoised
plt.subplot(2, 3, 4)
plt.imshow(denoised_salt, cmap='gray')
plt.title('Salt Denoised Image', fontsize=fs)
plt.axis('off')

# Pepper noise denoised
plt.subplot(2, 3, 5)
plt.imshow(denoised_pepper, cmap='gray')
plt.title('Pepper Denoised Image', fontsize=fs)
plt.axis('off')

# Wrongly denoised salt noise image
plt.subplot(2, 3, 6)
plt.imshow(wrong_denoised_salt, cmap='gray')
plt.title('Wrongly Denoised Salt', fontsize=fs)
plt.axis('off')

plt.tight_layout()
plt.show()


## 6.3 Noise low-pass filter (LPF)

In [ ]:
# Generate meshgrid for frequency responses
pi = np.pi
wx, wy = np.meshgrid(np.linspace(-pi, pi, 33), np.linspace(-pi, pi, 33))

# Select the compute type
# filter_type = '1d_horizontal'
# filter_type = '1d_vertical'
filter_type = '2d'

# Filter frequency-domain by type
H = 1 / 256
h = H
if filter_type == '2d':
    H = H * (1 + 2 * np.cos(wx) + 2 * np.cos(2 * wx)) * (1 + 2 * np.cos(wy) + 2 * np.cos(2 * wy))
    h = h * np.ones((16, 16))
elif filter_type == '1d_horizontal':
    H = H * (1 + 2 * np.cos(wx) + 2 * np.cos(2 * wx))
    h = np.ones((1, 16)) / 16
elif filter_type == '1d_vertical':
    H = H * (1 + 2 * np.cos(wy) + 2 * np.cos(2 * wy))
    h = np.ones((16, 1)) / 16

# Plot the frequency response
plt.figure(figsize=(8, 6))
ax = plt.axes(projection = '3d')
ax.plot_surface(wx / pi, wy / pi, np.abs(H), cmap = 'viridis')
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([0, 1])
ax.set_xticks(np.linspace(-1, 1, 5))
ax.set_yticks(np.linspace(-1, 1, 5))
ax.set_xlabel(r'$\omega_x / \pi$', fontsize = FS)
ax.set_ylabel(r'$\omega_y / \pi$', fontsize = FS)
ax.set_zlabel(r'$|H(\omega_x, \omega_y)|$', fontsize = FS)
plt.title('Frequency Response', fontsize = FS)
plt.show()

# Load the image
image = cv2.imread('airplane.png')

# Apply the low-pass filter
filtered_image = cv2.filter2D(image, -1, h, borderType = cv2.BORDER_REFLECT)

# Display the original and filtered images
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title('Original Image', fontsize = FS)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(filtered_image)
plt.title('Filtered Image', fontsize = FS)
plt.axis('off')

plt.tight_layout()
plt.show()

## 6.4 Image sharpening

In [ ]:
# Load image
# img_src = "duck.png"
img_src = "street.jfif"
image = cv2.imread(img_src)
image = image / 255.0  # Normalize to [0, 1]

# Define sharpening filters
h1 = np.array([[0, -1, 0], 
               [-1, 10, -1], 
               [0, -1, 0]]) / 4
h2 = np.array([[0, -1, 0], 
               [-1, 5, -1], 
               [0, -1, 0]])

# Apply sharpening filters
sharpened1 = cv2.filter2D(image, -1, h1, borderType = cv2.BORDER_REFLECT)
sharpened2 = cv2.filter2D(image, -1, h2, borderType = cv2.BORDER_REFLECT)

# Display results
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title('Original Image', fontsize = FS)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(sharpened1)
plt.title('Sharpened Image 1', fontsize = FS)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(sharpened2)
plt.title('Sharpened Image 2', fontsize = FS)
plt.axis('off')

plt.tight_layout()
plt.show()


## 6.5.1 Wiener noise filtering with additive noisy and blurry image

In [ ]:
# Define helper function for Mean Square Error
def mse(imageA, imageB):
    return np.mean((imageA - imageB) ** 2)

# Load image
I = cv2.imread('eagle.jfif', cv2.IMREAD_GRAYSCALE)
I = I / 255.0  # Normalize to [0, 1]

# Add blurry effect
LEN = 3
THETA = 5
PSF = cv2.getGaussianKernel(LEN, -1).dot(cv2.getGaussianKernel(LEN, -1).T)
blurry_I = convolve2d(I, PSF, mode = 'same', boundary = 'symm')

# Add noise effect
mean = 0
var = 0.01
noisy_blurred_I = blurry_I + np.random.normal(mean, np.sqrt(var), I.shape)

# Display images
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.imshow(I, cmap = 'gray')
plt.title('Original Image', fontsize=15)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(blurry_I, cmap = 'gray')
plt.title('Blurry Image', fontsize=15)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(noisy_blurred_I, cmap = 'gray')
plt.title('Noisy Blurry Image', fontsize=15)
plt.axis('off')

plt.tight_layout()
plt.show()

# Compare MSE before restoration
mse_before = mse(noisy_blurred_I, I)
print(f"MSE before restoration: {mse_before:.2f}")

# Step 1: Denoise the image using Wiener filter
wnr_denoise = restoration.wiener(noisy_blurred_I, PSF, balance=var)

# Step 2: Deblur the denoised image
sharpen_filter = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
wnr_deblur = convolve2d(wnr_denoise, sharpen_filter, mode = 'same', boundary = 'symm')

# Display restored images
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.imshow(noisy_blurred_I, cmap = 'gray')
plt.title('Noisy Blurry Image', fontsize=15)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(wnr_denoise, cmap = 'gray')
plt.title('Denoised Blurry Image', fontsize=15)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(wnr_deblur, cmap = 'gray')
plt.title('Denoised & Deblurred Image', fontsize=15)
plt.axis('off')

plt.tight_layout()
plt.show()

# Compare MSE after restoration
mse_after = mse(wnr_deblur, I)
print(f"MSE after restoration: {mse_after:.2f}")
